# C print
ArraySort(BitVecSort(), t)
Records
Enums
Tagged unions

Recrusive tagged unions are the thing that kind of needs something extra.

Pretty printing to rust might be nice. Could use Rc.
Could use python.h objects?

must_tail
fbip?


pretty print lean?
ocaml?
guile?
julia?


Could do a dag compression. But why bother? C compiler will do it for me. Just is more complex.

Tricks from pcode2c? special constants.
including gmp

include a gc, or just malloc and never free?

I could thread a state object?
Some special IO monad


You know, honestly a big struct is easier to deal with than an array.



In [2]:
from kdrag.all import *
def finsize(s : smt.SortRef):
    if isinstance(s, smt.BitVecSortRef):
        return True
def ctype_of_sort(s : smt.SortRef):
    if s == smt.BoolSort():
        return "bool"
    if isinstance(s, smt.BitVecSortRef):
        if s.size() in [8, 16, 32, 64]:
            return f"uint{s.size()}_t"
        else:
            raise NotImplementedError("No support for arbitrary C int sizes", s.size())
    else:
        raise NotImplementedError(f"Cannot convert {s} to C type")

import re

# Regex pattern for valid C identifiers
c_identifier_regex = r"^[A-Za-z_][A-Za-z0-9_]*$"

# Function to check if a given string is a valid C identifier
def is_valid_c_identifier(identifier: str) -> bool:
    return bool(re.match(c_identifier_regex, identifier))

# List of C keywords (C11 standard)
c_keywords = {
    "auto", "break", "case", "char", "const", "continue", "default", "do",
    "double", "else", "enum", "extern", "float", "for", "goto", "if",
    "inline", "int", "long", "register", "restrict", "return", "short",
    "signed", "sizeof", "static", "struct", "switch", "typedef", "union",
    "unsigned", "void", "volatile", "while", "_Alignas", "_Alignof",
    "_Atomic", "_Bool", "_Complex", "_Generic", "_Imaginary", "_Noreturn",
    "_Static_assert", "_Thread_local"
}

# Function to check valid C identifier considering keywords
def is_valid_c_identifier_strict(identifier: str) -> bool:
    return is_valid_c_identifier(identifier) and identifier not in c_keywords



binops = {
    "bvadd": "+",
    "bvsub": "-",
    "bvmul": "*",
}

comp = {
    "bvuge": ">=",
    "bvult": "<",
    "bvule": "<=",
    "bvugt": ">",
}

def c_of_expr(ctx : list[smt.ExprRef], sig : list[smt.FuncDeclRef], e : smt.ExprRef) -> str:
    ctype_of_sort(e.sort()) # check sort is supported
    if any(e.eq(c) for c in ctx):
        assert is_valid_c_identifier_strict(e.decl().name())
        return e.decl().name()
    elif smt.is_app(e):
        decl = e.decl()
        decl_name = decl.name()
        children = [c_of_expr(ctx, sig, c) for c in e.children()]
        assert all(isinstance(c, str) for c in children)
        nargs = len(children)
        if e.decl() in sig:
            assert is_valid_c_identifier_strict(e.decl().name())
            return f"{e.decl().name()}({", ".join(children)})"
        elif smt.is_if(e):
            return f"({children[0]} ? {children[1]} : {children[2]})"
        elif isinstance(e, smt.BoolRef):
            if smt.is_true(e):
                return "true"
            elif smt.is_false(e):
                return "false"
            elif smt.is_and(e):
                return f"({' && '.join(children)})"
            elif smt.is_or(e):
                return f"({' || '.join(children)})"
            elif smt.is_not(e):
                assert nargs == 1
                return f"(!{c_of_expr(ctx, children[0])})"
            elif decl_name in comp:
                assert nargs == 2
                return f"({children[0]} {comp[decl_name]} {c_of_expr(ctx, sig, e.children()[1])})"
            elif decl_name in ["bvsge", "bvslt", "bvsle", "bvsgt"]:
                raise NotImplementedError("Signed comparison not supported")
            else:
                raise NotImplementedError("Unsupported boolean expression", e)
        elif isinstance(e, smt.BitVecRef):
            if smt.is_bv_value(e):
                return "0b" + e.as_binary_string()
            elif nargs == 1:
                if decl_name == "bvnot":
                    return f"(~{children[0]})"
                elif decl_name == "bvneg":
                    return f"(-{children[0]})"
                else:
                    raise NotImplementedError(f"Unsupported unary operation: {decl_name}")
            if nargs == 2:
                op = binops.get(decl_name)
                if op is not None:
                    return f"({children[0]} {op} {children[1]})"
                elif decl_name in ["|", "&", "^"]:
                    return f"({children[0]} {decl_name} {children[1]})"
                else:
                    raise NotImplementedError(f"Unsupported binary operation: {decl_name}")
            else:
                raise NotImplementedError(f"Unsupported bitvector operation with {nargs} arguments: {decl_name}")
        else:
            raise NotImplementedError(f"Unsupported expression type: {e}")
        # TODO: floats
    else:
        raise NotImplementedError(f"Unsupported expression type: {e}")


def cstring(name, args, body):
    assert is_valid_c_identifier_strict(name)
    assert all(is_valid_c_identifier_strict(arg.decl().name()) for arg in args)
    assert all(smt.is_const(arg) for arg in args)
    decl = smt.Function("name", *([arg.sort() for arg in args] + [body.sort()]))
    return f"""\
#include <stdint.h>
#include <stdbool.h>
{ctype_of_sort(body.sort())} {name}({', '.join([f"{ctype_of_sort(arg.sort())} {arg.decl().name()}" for arg in args])}){{
    return {c_of_expr(args, [decl], body)};
}}
"""

#collect_sorts
x,y = smt.BitVecs("x y", 64)
print(cstring("foo", [x,y], smt.If(smt.UGT(x + x*y + 1, x), x , y)))


#include <stdint.h>
#include <stdbool.h>
uint64_t foo(uint64_t x, uint64_t y){
    return ((((x + (x * y)) + 0b1) > x) ? x : y);
}



In [55]:
(x + x*y + 17).children()[1].as_string()

'17'

In [3]:
import subprocess
import tempfile
import os
def compile_c(c_code):
    tmpdir = tempfile.mkdtemp() #with tempfile.TemporaryDirectory(delete=False) as tmpdir:
    c_file = os.path.join(tmpdir, "temp.c")
    so_file = os.path.join(tmpdir, "temp.so")

    with open(c_file, "w") as f:
        f.write(c_code)

    # Compile C code into a shared object
    subprocess.run(["gcc", "-Wall", "-Wextra", "-Wconversion", "-Warith-conversion", "-Werror" , "-shared", "-fPIC", c_file, "-o", so_file], check=True, capture_output=True)
    return so_file
      
compile_c(cstring("foo", [x,y], smt.If(smt.UGT(x + x*y + 1, x), x , y)))

'/tmp/tmpznk7nuwe/temp.so'

In [4]:
import cffi
def link(name, args, body, filename):
    ffi = cffi.FFI()
    ffi.cdef(f"""\
    {ctype_of_sort(body.sort())} {name}({', '.join([f"{ctype_of_sort(arg.sort())}" for arg in args])});
    """)
    lib = ffi.dlopen(filename)
    return lib

def compile_and_link(name, args, body):
    so_file = compile_c(cstring(name, args, body))
    return link(name, args, body, so_file)

lib = compile_and_link("mymax", [x,y], smt.If(smt.UGT(x, y), x, y))
lib.mymax(3,42)

42

In [88]:
%%file /tmp/test.c
#include <Python.h>
int main(int argc, char *argv[]){
    Py_Initialize();
    PyRun_SimpleString("print('Hello from Python')");
    Py_Finalize();
    return 0;
}


Overwriting /tmp/test.c


In [95]:
!gcc   /tmp/test.c  -o /tmp/test $(python3-config --includes --ldflags --embed) && /tmp/test

Hello from Python


In [96]:
%%file /tmp/test.c
#include <Python.h>
void hello(){
    //Py_Initialize();
    PyRun_SimpleString("print('Hello from Python')");
    //Py_Finalize();
    return; //return 0;
}

Overwriting /tmp/test.c


In [98]:
!gcc -fPIC -shared  /tmp/test.c  -o /tmp/test.so $(python3-config --includes --ldflags)

In [ ]:
import cffi

ffi = cffi.FFI()
ffi.cdef("void hello();")
lib = ffi.dlopen("/tmp/test.so")
lib.hello()

: 

In [33]:
(x + x*y > x).decl().name()

'bvsgt'

In [ ]:
from kdrag.all import *
x,y = smt.BitVecs("x y", 64)
z,w = smt.BitVecs("z w", 32)
d1 = (z + w).decl()
d2 = (x + y).decl()
d1 == d2

smt.UGE(x , y).decl().name()
{
(~x).decl().name() : "~"
(-x).decl().name() : "-"
}

'bvneg'

In [6]:
smt.is_add(z + w)

False

# Rust



In [ ]:
import tempfile
import subprocess

def compile_rust(rust_code):
    tmpdir = tempfile.mkdtemp()
    subprocess.run(["cargo", "init", "--lib", tmpdir], check=True, capture_output=True)
    subprocess.run(["cargo", "add", "pyo3", "--features", "extension-module", "--manifest-path", tmpdir], check=True, capture_output=True)
    rs_file = os.path.join(tmpdir, "src", "temp.rs")
    with open(rs_file, "w") as f:
        f.write(rust_code)
    # Compile Rust code into a shared object
    subprocess.run(["maturin-develop", "-m", os.path.join(tmpdir, "Cargo.toml")], check=True, capture_output=True)

In [ ]:
def gen_rust(name, args, body):
    return f"""\
use pyo3::prelude::*;
use pyo3::wrap_pyfunction;

#[pyfunction]
fn {name}({",".join([arg.decl().name() + ":" + arg.sort() for arg in args])}) -> {body.sort()} {{
    {rust_of_expr(body)}
}}

#[pymodule]
fn {name}(py: Python, m: &PyModule) -> PyResult<()> {{
    m.add_function(wrap_pyfunction!({name}, m)?)?;
    Ok(())
}}
"""

In [ ]:
!cargo new --lib /tmp/temp_rust_project
!cargo add pyo3 --features extension-module --manifest-path /tmp/temp_rust_project/Cargo.toml

In [ ]:
%%file /tmp/temp_rust_project/src/lib.rs
use pyo3::prelude::*;

#[pyfunction]
fn sum_array(arr: Vec<f64>) -> f64 {
    arr.iter().sum()
}

// fn guessing_game(m: &Bound<'_, PyModule>) -> PyResult<()>
#[pymodule]
fn temp_rust_project(m: &Bound<'_, PyModule>) -> PyResult<()>{
    m.add_function(wrap_pyfunction!(sum_array, m)?)?;
    Ok(())
}

In [ ]:
!maturin develop -m /tmp/temp_rust_project/Cargo.toml

In [ ]:
import temp_rust_project
temp_rust_project.sum_array([1.0, 2.0, 3.0, 4.0])

# wasm

Reference wasm interpreter
https://github.com/WebAssembly/spec/blob/main/interpreter/syntax/ast.ml
https://github.com/WebAssembly/spec/tree/main/interpreter/exec

https://www.certora.com/blog/formally-verifying-webassembly hmm.

In [ ]:
from kdrag.all import *
# deep embed
"""
  type unop = Clz | Ctz | Popcnt | ExtendS of pack_size
  type binop = Add | Sub | Mul | DivS | DivU | RemS | RemU
             | And | Or | Xor | Shl | ShrS | ShrU | Rotl | Rotr
  type testop = Eqz
  type relop = Eq | Ne | LtS | LtU | GtS | GtU | LeS | LeU | GeS | GeU
  type cvtop = ExtendSI32 | ExtendUI32 | WrapI64
             | TruncSF32 | TruncUF32 | TruncSF64 | TruncUF64
             | TruncSatSF32 | TruncSatUF32 | TruncSatSF64 | TruncSatUF64
             | ReinterpretFloat"
"""
IntBinOp = kd.Enum("IntBinOp", ["Add", "Sub", "Mul"])

Var = smt.BitVecSort(32)

Instr = kd.Inductive("Instr")
for n in ["Unreachable", "Nop", "Drop"]:
    Instr.declare(n)
Instr.declare("")


Looks like I might need mutually recursive datatypes. That's annoying.



In [ ]:
import kdrag.theories.list as list_


Frame = kd.Struct("Frame", ("inst", ModuleInst), ("locals", list_.List(Var)))
AdminInstr = kd.Inductive("AdminInstr")

Config = kd.Struct("Config", [("frame", Frame), ("code", )])
kd.define("step", [],m )

https://bytecodealliance.github.io/wasmtime-py/

In [ ]:
import wasmtime




# llvmlite

pretty print mlir
https://github.com/sdiehl/mlir-egglog
https://www.stephendiehl.com/posts/mlir_memory/


In [65]:
from llvmlite import ir

double = ir.DoubleType()
fnty = ir.FunctionType(double, (double, double))
module = ir.Module()
foo = ir.Function(module, fnty, name="foo")
bb = foo.append_basic_block("entry")
builder = ir.IRBuilder(bb)
a, b = foo.args
result = builder.fadd(a, b)
builder.ret(result)
print(module)

; ModuleID = ""
target triple = "unknown-unknown-unknown"
target datalayout = ""

define double @"foo"(double %".1", double %".2")
{
entry:
  %".4" = fadd double %".1", %".2"
  ret double %".4"
}



# verilog